In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/street-food-image-classification/sample_submission.csv
/kaggle/input/street-food-image-classification/train.csv
/kaggle/input/street-food-image-classification/test.csv
/kaggle/input/street-food-image-classification/train_images/falafel/0029_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0079_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0069_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0081_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0095_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0092_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0025_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0173_falafel.jpg
/kaggle/input/street-food-image-classification/train_images/falafel/0034_falafel.jpg
/kaggle/input/street-food-image-classification/train_

In [119]:
import torch
from torch import nn
from torch.optim import Adam
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import torchvision.models as models

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm,trange
from PIL import Image
from pathlib import Path

In [120]:
train_csv = pd.read_csv('/kaggle/input/street-food-image-classification/train.csv')
test_csv = pd.read_csv('/kaggle/input/street-food-image-classification/test.csv')

In [121]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_csv['label_encoded'] = le.fit_transform(train_csv['label'])

In [122]:
classes = train_csv['label'].unique()
print(classes)

['burger' 'churros' 'crepes' 'falafel' 'hot_dog' 'pad_thai' 'pani_puri'
 'pretzel' 'shawarma' 'tacos']


In [123]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    train_csv,
    test_size=0.2,
    random_state=42,
    stratify=train_csv['label_encoded']
)

In [124]:
train_dir = Path('/kaggle/input/street-food-image-classification/train_images')
test_dir = Path('/kaggle/input/street-food-image-classification/test_images')

In [125]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [126]:
class FoodDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['image_id']
        label = self.dataframe.iloc[idx]['label_encoded']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [127]:
train_dataset = FoodDataset(dataframe=train_df, image_dir=train_dir, transform=transform)
val_dataset   = FoodDataset(dataframe=val_df, image_dir=train_dir, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [128]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [129]:
model = models.resnet50(pretrained=True)
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [131]:
for param in model.parameters():
  # set to true unfreeze layers
  param.requires_grad = False

In [132]:
num_classes = len(train_csv['label'].unique())
model.fc = nn.Linear(model.fc.in_features, num_classes)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [133]:
criterion = nn.CrossEntropyLoss()
optimizier = Adam(model.parameters(),lr=0.001)

In [134]:
from tqdm import tqdm

class Trainer():
  def __init__(self,model,train_dataloader,test_dataloader,criterion,optimizier,epochs=3):
    self.model = model
    self.train_dataloader = train_dataloader
    self.test_dataloader = test_dataloader
    self.criterion = criterion
    self.optimizier = optimizier

    # epochs & history
    self.epochs = epochs
    self.history = {'loss':[], 'acc':[], 'val_loss':[], 'val_acc':[]}

  def train_loop(self):

    sum_loss = 0
    sum_accuracy = 0

    n = len(self.train_dataloader)

    for i, (data,label) in enumerate(tqdm(self.train_dataloader)):
      data = data.to(device)
      label = label.to(device).long()
      # prediction
      output = self.model(data)
      loss = self.criterion(output,label)

      sum_loss+=loss.item()
      n_corrects = (output.argmax(axis=1)==label).sum().item()
      sum_accuracy += n_corrects/label.size(0)

      loss.backward()
      self.optimizier.step()
      self.optimizier.zero_grad()

    train_loss = sum_loss/n
    train_accuracy = sum_accuracy/n

    self.history['loss'].append(train_loss)
    self.history['acc'].append(train_accuracy)

    return train_loss,train_accuracy

  def validation_loop(self):
    sum_loss = 0
    sum_accuracy = 0

    n = len(self.test_dataloader)

    for i, (data,label) in enumerate(tqdm(self.test_dataloader)):
      data = data.to(device)
      label = label.to(device).long()
      # prediction
      output = self.model(data)
      loss = self.criterion(output,label)
      n_corrects = (output.argmax(axis=1)==label).sum().item()

      sum_loss+=loss.item()
      sum_accuracy += n_corrects/label.size(0)

      # loss.backward()

    val_loss = sum_loss/n
    val_accuracy = sum_accuracy/n

    self.history['val_loss'].append(val_loss)
    self.history['val_acc'].append(val_accuracy)

    return val_loss,val_accuracy

  def train(self):

    for epoch in range(self.epochs):

      train_loss, train_acc = self.train_loop()
      val_loss, val_acc = self.validation_loop()
      print()
      print(f"Epoch{epoch+1}/{self.epochs} \t train_loss: {train_loss:.5}, train_acc: {train_acc:.5}, \t train_loss: {val_loss:.5}, train_acc: {val_acc:.5},")


In [135]:
epochs = 10
trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=val_dataloader,  # <-- This is now your validation set
    criterion=criterion,
    optimizier=optimizier,
    epochs=epochs
)


In [136]:
trainer.train()

100%|██████████| 10/10 [00:01<00:00,  5.28it/s]



Epoch1/10 	 train_loss: 1.8474, train_acc: 0.41776, 	 train_loss: 1.4048, train_acc: 0.59375,


100%|██████████| 10/10 [00:01<00:00,  5.84it/s]



Epoch2/10 	 train_loss: 1.0763, train_acc: 0.71957, 	 train_loss: 1.1124, train_acc: 0.68437,


100%|██████████| 10/10 [00:01<00:00,  5.81it/s]


...


Epoch8/10 	 train_loss: 0.48877, train_acc: 0.85691, 	 train_loss: 0.69924, train_acc: 0.75,


100%|██████████| 10/10 [00:01<00:00,  5.98it/s]



Epoch9/10 	 train_loss: 0.45888, train_acc: 0.8676, 	 train_loss: 0.6884, train_acc: 0.77812,


100%|██████████| 10/10 [00:01<00:00,  6.06it/s]


Epoch10/10 	 train_loss: 0.4128, train_acc: 0.87747, 	 train_loss: 0.67468, train_acc: 0.77812,


## submission

In [137]:
test_dataset = TestFoodDataset(dataframe=test_csv, image_dir=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [138]:
model.eval()  # set model to evaluation mode
predictions = []
image_ids = []

with torch.no_grad():
    for images, ids in tqdm(test_loader):
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        predictions.extend(preds)
        image_ids.extend(ids)

100%|██████████| 24/24 [00:04<00:00,  5.23it/s]


In [139]:
predicted_labels = le.inverse_transform(predictions)

In [140]:
submission = pd.DataFrame({
    'image_id': image_ids,
    'label': predicted_labels
})

submission.to_csv('submission.csv',index=False)